In [0]:
import tensorflow as tf
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from tensorflow.keras import layers
import os
li_text=[]
all_words=[]
#li_files=os.listdir('D:/dataset/New folder/HCL ML Challenge/HCL ML Challenge Dataset')


In [0]:
results0=pd.read_csv('D:/dataset/New folder/HCL ML Challenge/results0.csv')
li_files=results0['Filename']

In [0]:
sam1=open('D:/dataset/New folder/HCL ML Challenge/Sample Dataset/sample2_0000001R.txt','r').read()
sam0=open('D:/dataset/New folder/HCL ML Challenge/Sample Dataset/sample1_0000001R.txt','r').read()
out=pd.read_csv('D:/dataset/New folder/HCL ML Challenge/sample_submission.csv')
out_0=out.values[0][1]

out_1=out.values[1][1]
pun=['{','.','nan',':',',',"'",'``', "''", '``','}','Â£']


def featurise(doc,out):
    doc=doc.replace(',','')
    pun=['{','.','nan',':',',',"'",'``', "''", '``','}','Â£']
    doc=word_tokenize(doc)
    out=word_tokenize(out)
    doc_pun=[]
    for i in doc:
        if i not in pun:
            doc_pun.append(i)
    out_pun=[]
    for i in out:
        if i not in pun:
            out_pun.append(i)
    dat=[]

    for i in doc_pun:
        if i in out_pun:
            dat.append(1)
        else:
            dat.append(0)
    return pd.DataFrame({'word':doc_pun,"present":dat})

table1=featurise(sam1,out_1)
table0=featurise(sam0,out_0)
table1.to_csv('D:/dataset/New folder/HCL ML Challenge/table1.csv')
table0.to_csv('D:/dataset/New folder/HCL ML Challenge/table0.csv')

In [0]:
for i in li_files:
    f=open('D:/dataset/New folder/HCL ML Challenge/HCL ML Challenge Dataset/{}.txt'.format(i),'r')
    
    st=f.read()
    st=st.replace(',','')
    st=word_tokenize(st)
    pun=['{','.','nan',':',',',"'",'``', "''", '``','}','Â£']
    st_pun=[]
    for i in st:
        if i not in pun:
            st_pun.append(i)
            if (i.isnumeric() != True)|(i=='2019'):
                all_words.append(i)
                
    li_text.append(st_pun)

In [0]:
all_words=list(nltk.FreqDist(all_words).keys())

table0=pd.read_csv('D:/dataset/New folder/HCL ML Challenge/table0.csv')
table1=pd.read_csv('D:/dataset/New folder/HCL ML Challenge/table1.csv')

seq_len=10
vocab_len=len(all_words)

In [0]:
def featurise(doc):
    feature=[]
    for i in doc:
        if (i.isnumeric())&(i!='2019'):
            feature.append(vocab_len)
        else:
            feature.append(all_words.index(i))
    return feature

features=[]
for doc in li_text:
    feat_text=featurise(doc)
    features.append(feat_text)

In [0]:
seq_feature=[]
for doc in features:
    temp=[]
    for i in range(0,len(doc)-10+1):
        seq=[]
        for j in range(0,seq_len):
            seq.append(doc[j+i])
        temp.append(seq)
    seq_feature.append(temp)
    
seq_lable=[table0['present'][seq_len-1:],table1['present'][seq_len-1:]]
len(seq_lable[0])
#len(seq_lable[1])

276

In [0]:
model=tf.keras.Sequential([
    layers.Embedding(vocab_len+1,256,input_length=10),
    layers.LSTM(1024,return_sequences=True),
    layers.LSTM(1024),
    layers.Dense(2,activation='softmax')
    ])

In [0]:
z=list(seq_lable[0])
for i in seq_lable[1]:
  z.append(i)


In [0]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(np.array(seq_feature[0]+seq_feature[1]),np.array(z),epochs=40)

Train on 518 samples
Epoch 1/40
518/518 [==============================] - 2s 4ms/sample - loss: 0.4412 - accuracy: 0.7992
Epoch 2/40
518/518 [==============================] - 0s 716us/sample - loss: 0.4104 - accuracy: 0.8822
Epoch 3/40
518/518 [==============================] - 0s 722us/sample - loss: 0.2867 - accuracy: 0.8764
Epoch 4/40
518/518 [==============================] - 0s 724us/sample - loss: 0.2609 - accuracy: 0.8919
Epoch 5/40
518/518 [==============================] - 0s 705us/sample - loss: 0.2190 - accuracy: 0.9015
Epoch 6/40
518/518 [==============================] - 0s 709us/sample - loss: 0.1782 - accuracy: 0.9112
Epoch 7/40
518/518 [==============================] - 0s 724us/sample - loss: 0.1560 - accuracy: 0.9305
Epoch 8/40
518/518 [==============================] - 0s 710us/sample - loss: 0.1224 - accuracy: 0.9440
Epoch 9/40
518/518 [==============================] - 0s 706us/sample - loss: 0.1069 - accuracy: 0.9595
Epoch 10/40
518/518 [========================

In [0]:
model.save('model1_hcl.hdf5')

In [0]:
import json


def predict_values1(text_file_no):
  a=model.predict(seq_feature[text_file_no])
  v=0
  t=9
  output_text=[]
  for i in a:
    if i[0]<i[1]:
      f=seq_feature[text_file_no][t-9][9]
      if f!=len(all_words):
        output_text.append(all_words[f])
      else:
        output_text.append(li_text[text_file_no][t])
    v+=1
    t+=1
  #print(output_text)
  
  temp=''
  output='{'
  flag=0
  for i in output_text:
    if i.isnumeric()!=True and i!='(' and i!=')':
      temp+=i+' '
    elif i=='(':
      flag=1
    elif i==')':
      flag=0
    else:
      if flag==0:
        temp=temp[:len(temp)-1]
        output+='"'+temp+'"'+':'+'"'+i+'"'+','
        temp=''
      elif flag==1:
        temp=temp[:len(temp)-1]
        output+='"'+temp+'"'+':'+'"-'+i+'"'+','
        temp=''
  



  output=output[:len(output)-1]
  output+='}'
  #return output
  return json.loads(output)


predict_values1(1)

{'Fixed assets Intangible assets': '9423',
 'Tangible assets': '6414',
 'Current assets Debtors amounts falling due within one year': '113831',
 'Cash at bank and in hand': '122352',
 '': '70587',
 'Creditors amounts falling due within one year': '110924',
 'Net current assets': '125259',
 'Net assets': '141096',
 'Capital and reserves Called up share capital': '70587',
 'Profit and loss account': '70509',
 'Shareholders funds': '141096'}

In [0]:
res=[]
for i in range(2,502):
  res.append(predict_values1(i))
  

results=pd.DataFrame({'Filename':li_files[2:],'Extracted Values':res})
results.to_csv('C:/Users/ninad/Desktop/Results.csv',index=False)

In [0]:
results=pd.DataFrame({'Filename':li_files[2:],'Extracted Values':res})
results.to_csv('C:/Users/ninad/Desktop/Results0.csv',index=False)